In [ ]:
!dotnet new console -o consumeModelApp

In [ ]:
!mlnet auto-train --task binary-classification --dataset "wikipedia-detox-250-line-data.tsv" --label-column-name "Sentiment" --max-exploration-time 10

In [ ]:
!cd consumeModelApp && dotnet add reference ../SampleBinaryClassification/SampleBinaryClassification.Model/

In [ ]:
!cd consumeModelApp && dotnet add package Microsoft.ML --version 1.4.0

In [ ]:
!cp ./SampleBinaryClassification/SampleBinaryClassification.Model/MLModel.zip ./consumeModelApp

In [ ]:
%%writefile ./consumeModelApp/Program.cs
using System;
using SampleBinaryClassification.Model.DataModels;
using Microsoft.ML;

namespace consumeModelApp
{
    class Program
    {
        static void Main(string[] args)
        {
            ConsumeModel();
        }

        public static void ConsumeModel()
        {
            // Load the model
            MLContext mlContext = new MLContext();

            ITransformer mlModel = mlContext.Model.Load("MLModel.zip", out var modelInputSchema);

            var predEngine = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(mlModel);

            // Use the code below to add input data
            var input = new ModelInput();
            input.SentimentText = "That is rude";

            // Try model on sample data
            // True is toxic, false is non-toxic
            ModelOutput result = predEngine.Predict(input);

            Console.WriteLine($"Text: {input.SentimentText} | Prediction: {(Convert.ToBoolean(result.Prediction) ? "Toxic" : "Non-toxic")} sentiment");
        }
    }
}

In [ ]:
!cd ./consumeModelApp && dotnet run

In [ ]:
!rm -rf ./consumeModelApp ./SampleBinaryClassification